In [5]:
from dataclasses import dataclass
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import numpy as np

data = pd.read_excel('/Users/gim-yeon-u/Desktop/구단홈구장 별 거리별 상권/상권 정보/원주 1km.xlsx')

data = data.loc[data['상권업종대분류명'] == '음식']  

data = data[['상호명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명', '행정동명', '위도', '경도']]


data.columns = ['name',  # 상호명
              'cate_1',  # 중분류명
              'cate_2',  # 소분류명
              'cate_3',  # 표준산업분류명
              'dong',  # 행정동명
              'lon',  # 위도
              'lat'  # 경도
              ]


driver = webdriver.Chrome('/Users/gim-yeon-u/Downloads/chromedriver')

data['name'] = list(map(str,data['name']))


data['naver_keyword'] = data['dong'] + "%20" + data['name']  # "%20"는 띄어쓰기를 의미합니다.
data['naver_map_url'] = ''


# 본격적으로 가게 상세페이지의 URL을 가져옵시다

for i, keyword in enumerate(data['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {data.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(3.5)
        data.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR, "#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                data.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR, "#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                data.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()


# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

data['naver_map_url'] = "https://m.place.naver.com/restaurant/" + data['naver_map_url']


# URL이 수집되지 않은 데이터는 제거합니다.
data = data.loc[~data['naver_map_url'].isnull()]

/Users/gim-yeon-u/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:31: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


이번에 찾을 키워드 : 0 / 1028 행 단구동%20유나인
이번에 찾을 키워드 : 1 / 1028 행 무실동%20한지테마파크카페테리아
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x00000001003aaf44 chromedriver + 3813188
1   chromedriver                        0x000000010033cae0 chromedriver + 3361504
2   chromedriver                        0x0000000100040c5c chromedriver + 232540
3   chromedriver                        0x0000000100072380 chromedriver + 435072
4   chromedriver                        0x000000010009ff44 chromedriver + 622404
5   chromedriver                        0x0000000100067358 chromedriver + 389976
6   chromedriver                        0x0000000100068284 chromedriver + 393860
7   chromedriver                        0x000000010037ee18 chromedriver + 3632664
8   chr

In [6]:
data['naver_map_url']

14        https://m.place.naver.com/restaurant/12998386
27      https://m.place.naver.com/restaurant/1689588139
28        https://m.place.naver.com/restaurant/15181020
38        https://m.place.naver.com/restaurant/15276825
43      https://m.place.naver.com/restaurant/1391634592
                             ...                       
3011              https://m.place.naver.com/restaurant/
3012              https://m.place.naver.com/restaurant/
3013              https://m.place.naver.com/restaurant/
3015              https://m.place.naver.com/restaurant/
3018              https://m.place.naver.com/restaurant/
Name: naver_map_url, Length: 1010, dtype: object

In [7]:
from tqdm.notebook import tqdm

naver_map_name_list = []
naver_map_type_list = []
blog_review_list = []
blog_review_qty_list = []
naver_map_star_review_stars_list = []
naver_map_star_review_qty_list = []
chromedriver = '/Users/gim-yeon-u/Downloads/chromedriver'

# 메인 드라이버 : 별점 등을 크롤링
driver = webdriver.Chrome(chromedriver) 

# 서브 드라이버 : 블로그 리뷰 텍스트를 리뷰 탭 들어가서 크롤링
sub_driver = webdriver.Chrome(chromedriver)

for i, url in enumerate(tqdm(data['naver_map_url'])):

    driver.get(url)
    sub_driver.get(url+"/review/ugc")
    time.sleep(2)

    try:

        # 간단 정보 가져오기
        
        # 네이버 지도의 유형 분류
        naver_map_type = driver.find_element(By.CSS_SELECTOR, "#_title > span.DJJvD").text

        # 블로그 리뷰 수
        blog_review_qty = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(3) > a > em").text

        # 블로그 별점 점수
        star_review_stars = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span.PXMot.LXIwF > em").text

        # 블로그 별점 평가 수
        star_review_qty = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(2) > a > em").text
       

        # 블로그 리뷰 텍스트 가져오기
        review_text_list = [] # 임시 선언
        
        # 네이버 지도 블로그 리뷰 탭은 동적 웹사이트의 순서가 주문하기, 메뉴보기 등의 존재 여부로 다르기 때문에 css selector가 아니라 element 찾기로 진행
        review_text_crawl_list = sub_driver.find_elements(By.CLASS_NAME, "span.veBoZ")
        
        # find element's' 메소드를 통해 가져온 내용은 리스트로 저장되고, 리스트 타입을 풀어서(for문 사용) 임시 데이터에 모아 두어야 한다
        for review_crawl_data in review_text_crawl_list:
            review_text_list.append(review_crawl_data.find_element(By.TAG_NAME, 'div').text)
        
        # 그 리스트에 저장된 텍스트 (한 식당에 대한 여러 리뷰들)를 한 텍스트 덩어리로 모아(join)줍니다.
        review_text = ','.join(review_text_list)


        blog_review_list.append(review_text)

        naver_map_type_list.append(naver_map_type)
        blog_review_qty_list.append(blog_review_qty)
        naver_map_star_review_stars_list.append(star_review_stars)
        naver_map_star_review_qty_list.append(star_review_qty)
        
        print(f"{i}row Success")

    # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
    except Exception as e1:
        print(f"{i}row Failure")
        
        # 리뷰가 없으므로 null을 임시로 넣어줍니다.
        blog_review_list.append('null')  
        naver_map_type_list.append('null')
        blog_review_qty_list.append('null')
        naver_map_star_review_stars_list.append('null')
        naver_map_star_review_qty_list.append('null')


        
driver.quit()
sub_driver.quit()


data['naver_store_type'] = naver_map_type_list  # 네이버 상세페이지에서 크롤링한 업체 유형
data['naver_star_point'] = naver_map_star_review_stars_list  # 네이버 상세페이지에서 평가한 별점 평점
data['naver_star_point_qty'] = naver_map_star_review_qty_list  # 네이버 상세페이지에서 별점 평가를 한 횟수
data['naver_blog_review_txt'] = blog_review_list  # 네이버 상세페이지에 나온 블로그 리뷰 텍스트들
data['naver_blog_review_qty'] = blog_review_qty_list  # 네이버 상세페이지에 나온 블로그 리뷰의 총 개수

/Users/gim-yeon-u/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  if sys.path[0] == "":


KeyboardInterrupt: 

In [ ]:
data

name  cate_1       cate_2    cate_3  dong        lon         lat  \
4      커핀그루나루신천점  커피점/카페  커피전문점/카페/다방  비알콜 음료점업  잠실본동  37.510988  127.082767   
9            한앤둘  커피점/카페  커피전문점/카페/다방  비알콜 음료점업  잠실2동  37.516188  127.075928   
12         남원추어탕      한식        추어탕전문   한식 음식점업  대치2동  37.508489  127.066307   
17    커피니종합운동장역점  커피점/카페  커피전문점/카페/다방  비알콜 음료점업  잠실7동  37.510959  127.073334   
18        놀부유황오리  닭/오리요리       오리고기전문   한식 음식점업  잠실본동  37.505285  127.078257   
...          ...     ...          ...       ...   ...        ...         ...   
1759         서촌제      양식       돈가스전문점  서양식 음식점업  잠실2동  37.519202  127.076490   
1760         한국계      한식    한식/백반/한정식   한식 음식점업  잠실본동  37.510741  127.079954   
1761         태정각      중식     중국음식/중국집   중식 음식점업  잠실본동  37.505573  127.082822   
1765       전집전문전      한식         파전전문   한식 음식점업  잠실본동  37.509960  127.079138   
1773      곱창과순대국      한식     곱창/양구이전문   한식 음식점업  삼성1동  37.512158  127.065311   

          naver_keyword                                    naver_map_url  \
4      잠실본동%20커핀그루나루신천점    https://m.place.naver.com/restaurant/33586527   
9            잠실2동%20한앤둘    https://m.place.naver.com/restaurant/32872936   
12         대치2동%20남원추어탕    https://m.place.naver.com/restaurant/20624013   
17    잠실7동%20커피니종합운동장역점    https://m.place.naver.com/restaurant/32779383   
18        잠실본동%20놀부유황오리    https://m.place.naver.com/restaurant/33617768   
...                 ...                                              ...   
1759         잠실2동%20서촌제  https://m.place.naver.com/restaurant/1287192190   
1760         잠실본동%20한국계  https://m.place.naver.com/restaurant/1223172043   
1761         잠실본동%20태정각  https://m.place.naver.com/restaurant/1634826962   
1765       잠실본동%20전집전문전  https://m.place.naver.com/restaurant/1591689939   
1773      삼성1동%20곱창과순대국  https://m.place.naver.com/restaurant/1542496600   

     naver_store_type naver_star_point naver_star_point_qty  \
4                  카페             4.33                  518   
9                null             null                 null   
12                                                            
17                 카페             4.46                  369   
18                                                            
...               ...              ...                  ...   
1759              돈가스              4.5                   56   
1760               한식              4.8                  878   
1761              중식당             4.21                  179   
1765             요리주점             4.85                   15   
1773             null             null                 null   

     naver_blog_review_txt naver_blog_review_qty  
4                                             50  
9                     null                  null  
12                                                
17                                             3  
18                                                
...                    ...                   ...  
1759                                           9  
1760                                         342  
1761                                           5  
1765                                          34  
1773                  null                  null  

[515 rows x 14 columns]

In [ ]:
# 크롤링 에러가 떠서 'null'을 넣어 둔 데이터는 활용 의미가 없으므로 행 삭제를 해줘도 됩니다
data = data.loc[~(data['naver_store_type'].str.contains('null'))]

In [ ]:
data

name  cate_1       cate_2    cate_3  dong        lon         lat  \
4      커핀그루나루신천점  커피점/카페  커피전문점/카페/다방  비알콜 음료점업  잠실본동  37.510988  127.082767   
12         남원추어탕      한식        추어탕전문   한식 음식점업  대치2동  37.508489  127.066307   
17    커피니종합운동장역점  커피점/카페  커피전문점/카페/다방  비알콜 음료점업  잠실7동  37.510959  127.073334   
18        놀부유황오리  닭/오리요리       오리고기전문   한식 음식점업  잠실본동  37.505285  127.078257   
30           대관령      한식    한식/백반/한정식   한식 음식점업  대치2동  37.503867  127.065233   
...          ...     ...          ...       ...   ...        ...         ...   
1758         반길집    유흥주점     기타일반유흥주점  일반유흥 주점업  잠실본동  37.509481  127.084788   
1759         서촌제      양식       돈가스전문점  서양식 음식점업  잠실2동  37.519202  127.076490   
1760         한국계      한식    한식/백반/한정식   한식 음식점업  잠실본동  37.510741  127.079954   
1761         태정각      중식     중국음식/중국집   중식 음식점업  잠실본동  37.505573  127.082822   
1765       전집전문전      한식         파전전문   한식 음식점업  잠실본동  37.509960  127.079138   

          naver_keyword                                    naver_map_url  \
4      잠실본동%20커핀그루나루신천점    https://m.place.naver.com/restaurant/33586527   
12         대치2동%20남원추어탕    https://m.place.naver.com/restaurant/20624013   
17    잠실7동%20커피니종합운동장역점    https://m.place.naver.com/restaurant/32779383   
18        잠실본동%20놀부유황오리    https://m.place.naver.com/restaurant/33617768   
30           대치2동%20대관령  https://m.place.naver.com/restaurant/1982903433   
...                 ...                                              ...   
1758         잠실본동%20반길집  https://m.place.naver.com/restaurant/1115205265   
1759         잠실2동%20서촌제  https://m.place.naver.com/restaurant/1287192190   
1760         잠실본동%20한국계  https://m.place.naver.com/restaurant/1223172043   
1761         잠실본동%20태정각  https://m.place.naver.com/restaurant/1634826962   
1765       잠실본동%20전집전문전  https://m.place.naver.com/restaurant/1591689939   

     naver_store_type naver_star_point naver_star_point_qty  \
4                  카페             4.33                  518   
12                                                            
17                 카페             4.46                  369   
18                                                            
30             카페,디저트              4.8                  408   
...               ...              ...                  ...   
1758               한식                5                   46   
1759              돈가스              4.5                   56   
1760               한식              4.8                  878   
1761              중식당             4.21                  179   
1765             요리주점             4.85                   15   

     naver_blog_review_txt naver_blog_review_qty  
4                                             50  
12                                                
17                                             3  
18                                                
30                                            59  
...                    ...                   ...  
1758                                          22  
1759                                           9  
1760                                         342  
1761                                           5  
1765                                          34  

[409 rows x 14 columns]

In [ ]:
# 별점 평균, 수 같은 데이터 역시 스트링 타입으로 크롤링이 되었으므로 numeric으로 바꿔줍니다.
data[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']] = data[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']].apply(pd.to_numeric,errors = 'coerce')

In [ ]:
data = pd.DataFrame(data)

In [ ]:
data

name  cate_1       cate_2    cate_3  dong        lon         lat  \
4      커핀그루나루신천점  커피점/카페  커피전문점/카페/다방  비알콜 음료점업  잠실본동  37.510988  127.082767   
12         남원추어탕      한식        추어탕전문   한식 음식점업  대치2동  37.508489  127.066307   
17    커피니종합운동장역점  커피점/카페  커피전문점/카페/다방  비알콜 음료점업  잠실7동  37.510959  127.073334   
18        놀부유황오리  닭/오리요리       오리고기전문   한식 음식점업  잠실본동  37.505285  127.078257   
30           대관령      한식    한식/백반/한정식   한식 음식점업  대치2동  37.503867  127.065233   
...          ...     ...          ...       ...   ...        ...         ...   
1758         반길집    유흥주점     기타일반유흥주점  일반유흥 주점업  잠실본동  37.509481  127.084788   
1759         서촌제      양식       돈가스전문점  서양식 음식점업  잠실2동  37.519202  127.076490   
1760         한국계      한식    한식/백반/한정식   한식 음식점업  잠실본동  37.510741  127.079954   
1761         태정각      중식     중국음식/중국집   중식 음식점업  잠실본동  37.505573  127.082822   
1765       전집전문전      한식         파전전문   한식 음식점업  잠실본동  37.509960  127.079138   

          naver_keyword                                    naver_map_url  \
4      잠실본동%20커핀그루나루신천점    https://m.place.naver.com/restaurant/33586527   
12         대치2동%20남원추어탕    https://m.place.naver.com/restaurant/20624013   
17    잠실7동%20커피니종합운동장역점    https://m.place.naver.com/restaurant/32779383   
18        잠실본동%20놀부유황오리    https://m.place.naver.com/restaurant/33617768   
30           대치2동%20대관령  https://m.place.naver.com/restaurant/1982903433   
...                 ...                                              ...   
1758         잠실본동%20반길집  https://m.place.naver.com/restaurant/1115205265   
1759         잠실2동%20서촌제  https://m.place.naver.com/restaurant/1287192190   
1760         잠실본동%20한국계  https://m.place.naver.com/restaurant/1223172043   
1761         잠실본동%20태정각  https://m.place.naver.com/restaurant/1634826962   
1765       잠실본동%20전집전문전  https://m.place.naver.com/restaurant/1591689939   

     naver_store_type  naver_star_point  naver_star_point_qty  \
4                  카페              4.33                 518.0   
12                                  NaN                   NaN   
17                 카페              4.46                 369.0   
18                                  NaN                   NaN   
30             카페,디저트              4.80                 408.0   
...               ...               ...                   ...   
1758               한식              5.00                  46.0   
1759              돈가스              4.50                  56.0   
1760               한식              4.80                 878.0   
1761              중식당              4.21                 179.0   
1765             요리주점              4.85                  15.0   

     naver_blog_review_txt  naver_blog_review_qty  
4                                            50.0  
12                                            NaN  
17                                            3.0  
18                                            NaN  
30                                           59.0  
...                    ...                    ...  
1758                                         22.0  
1759                                          9.0  
1760                                        342.0  
1761                                          5.0  
1765                                         34.0  

[409 rows x 14 columns]

In [ ]:
data.to_csv('원주 1km 맛집 리스트.csv')